In [5]:
import numpy as np
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

# Loading your IBM Q account(s)
provider = IBMQ.load_account()

ibmqfactory.load_account:WARNING:2021-04-12 15:53:12,214: Credentials are already in use. The existing account in the session will be replaced.


In [3]:
#from openql import openql as ql
#import qxelarator


import random
from math import *
import os
import re
import math
import matplotlib.pyplot as plt
import numpy as np

In [11]:
def randStr(szA,sz):
    bias=1/szA #IMPROVE:addbiashere
    rbs= ''
    for i in range(0,sz):
        rn=random.random()
        for j in range(0,szA):
            if rn<(j+1)*bias:
                rbs=rbs+str(j)#IMPROVE:BCDversion
                break
    return rbs
#Generatesarandomstringoflength’sz’overthealphabetofsize


In [16]:
AS={'00','01','10','11'} #Alphabet set{0,1,2,3}:={A,C,G,T}for DNA Nucleotide bases
A=len(AS) #Alphabet size
N=11 #Reference Genome size
w=randStr(4,N) #Reference Genome(e.g.w=”22013213”)
M=3 #ShortReadsize
ans=2 #Known answer for testing
p=w[ans:ans+M] #ShortRead

In [18]:
s=ceil(log2(N-M)) 
Q_D=s*M  #Number of data qubits
Q_anc=1 #Number of ancilla qubits
anc=Q_D #Ancilla qubit id
total_qubits=Q_D+Q_anc

In [55]:
def QPM():
    print(w,p)
    config_fn=os.path.join('gateConfig.json')
    platform=ql.Platform('platform_none',config_fn)
    prog=ql.Program('qpm_a4',total_qubits,platform) 
    #Kernel1:Initialization
    qk1=ql.Kernel('QCirc1',platform) #PROBLEM AREA
    Circ1(qk1)
    #Kernel2:Oraclestomarkspecificcharacter
    qk2=ql.Kernel('QCirc2',platform)
    bfa = ''.join('1' if w[i] == '0' else '0' for i in range (len(w)))
    bfa = ''.join('1' if w[i] == '1' else '0' for i in range (len(w)))
    bfa = ''.join('1' if w[i] == '2' else '0' for i in range (len(w)))
    bfa = ''.join('1' if w[i] == '3' else '0' for i in range (len(w)))
    
    #Kernel3: Grover Amplitude Amplification
    qk3 = ql.Kernel('QCirc3',platform)
    Circ3(qk3,s,M)
    #Kernel4:Measurement
    qk4=ql.Kernel('QCirc4',platform)
    Circ4(qk4)
    
    #ConstructProgramfromKernels
    prog.add_kernel(qk1) #Initialise
    #for pi in range(0,M): #Alternate iteration method
    for r in range(0,int(sqrt(N-M+1))):
        pi=random.randint(0,M-1)
        if p[pi]=='0':
            Circ2(qk2,bfa,pi)
        elif p[pi]=='1':
            Circ2(qk2,bfc,pi)
        elif p[pi]=='2':
            Circ2(qk2,bfg,pi)
        else:
            Circ2(qk2,bft,pi)
    prog.add_kernel(qk2) #Conditionalkernelcall
    del qk2
    
    #IMPROVE:Kerneltoqubitloosebindingbeingdiscussed
    qk2=ql.Kernel('QCirc2',platform)
    prog.add_kernel(qk3)
#Inversionaboutmean
#prog.add_kernel(qk4)
#Uncommentifusingmeasurementbasedanalytics
    prog.compile()
    #showQasm()
    qx=qxelarator.QX()
    qx.set('test_output/qpm_a4.qasm')
    #ResultanalyticsusingInternalStateVector
    qx.execute()
    qxopt=qx.get_state()
    isv=[0]*(2**total_qubits)
    ptrn=re.compile('\(([+-]\d+.\d*),([+-]\d+[.\d*]?)\)\s[|]([0-1]*)>')
    for line in qxopt.splitlines():
        mtch=ptrn.search(line)
        if mtch!=None:
            ar=float(mtch.group(1))
            ac=float(mtch.group(2))
            state=int(mtch.group(3),2)
            isv[state]=ar**2+ac**2
    ploty=[0]*(2**s)
    for i in range(0,len(isv)):
        stot=format(i,'0'+str(total_qubits)+'b')[::-1]
        sopt=int(stot[0:s],2)
        ploty[sopt]=ploty[sopt]+isv[i]
    print("PMax:",np.amax(ploty))
    print("Index:",np.argmax(ploty))
    plt.plot(ploty)
    plt.ylabel('Probability')
    plt.xlabel('Solution space')
    plt.ylim([0,1])
    plt.show() 
    return




    


In [23]:
    #ResultanalyticsusingMeasurement
''' 
res=[0]*s
STT=1000
#Numberofquantumstatetomographytrials
true_counter=0
foriinrange(STT):
qx.execute()
res[0]=res[0]+qx.get_measurement_outcome(0)
res[1]=res[1]+qx.get_measurement_outcome(1)
res[2]=res[2]+qx.get_measurement_outcome(2)

index=’’.join(’1’ifres[i]>STT/2else ’0’foriinrange(s))
print(”Index:”,int(index,2))

'''

In [37]:
def Circ1(k):
    for Qi in range(0,total_qubits):#Initialiseallqubitsto|0>state
        k.prepz(Qi)
    for si in range(0,s):#Uniformsuperpositionofpossiblestartingpositions(answers)
        k.gate("h",[si])
    for Mi in range(0,M-1):
        for si in range(0,s):#Copypositionencodingtonextsetofs
            k.gate("cnot",[Mi*s+si,(Mi+1)*s+si])
        for si in range(0,s):#Incrementpositionencoding
            ic=(Mi+1)*s-(si+1)
            k.gate("x",[ic])#Invertedcontrol
            nc=[]
            for sj in range(ic,(Mi+1)*s):
                nc.append(sj)
            for sj in range((Mi+2)*s-1,s+ic-1,-1):
                nCX(k,nc,sj,anc)#Decomposemulti−controlledCNOT
            k.gate("x",[ic])#Uncomputeinvertedcontrol
    return


In [39]:
def Circ2(k,bf,q):
    for fi in range(0,len(bf)):#Encodeoraclefunction
        if bf[fi] == '1':
            fis=format(fi,'0'+str(s)+'b')
            for fisi in range(0,s):#Encodebinaryvalueoffunctioninput
                if fis[fisi]=='0':
                    k.gate("x",[q*s+fisi])
                    k.gate("h",[(q+1)*s-1])#CPhasetoCNOTconversion
                    nc=[]
            for qsi in range(q*s,(q+1)*s-1):
                nc.append(qsi)
                nCX(k,nc,(q+1)*s-1,anc)#Decomposemulti−controlledCNOT
                k.gate("h",[(q+1)*s-1])#UncomputeCPhasetoCNOTconversion
                for fisi in range(0,s):#Uncomputebinaryvalueoffunctioninput
                    if fis[fisi]=='0':
                        k.gate("x",[q*s+fisi])
    return

In [41]:
def Circ3(k,s,M):
    for si in range(0,s*M):
        k.gate("h",[si])
        k.gate("x",[si])
    k.gate("h",[s*M-1])
    nc=[]
    for sj in range(0,s*M-1):
        nc.append(sj)
    nCX(k,nc,s*M-1,s*M)
    k.gate("h",[s*M-1])
    for si in range(0,s*M):
        k.gate("x",[si])
        k.gate("h",[si])
    return

#CPhasetoCNOTconversion

#Decomposemulti−controlledCNOT
#UncomputeCPhasetoCNOTconversion


In [42]:
def Circ4(k):
    for si in range(0,s): #Measure first set of positions
        k.gate("measure",[si])
    return



In [44]:
def nCX(k,c,t,b):
    nc=len(c)
    if nc==1:
        k.gate("cnot",[c[0],t])
    elif nc==2:
        k.toffoli(c[0],c[1],t)
    else:
        nch=ceil(nc/2)
        c1=c[:nch]
        c2=c[nch:]
        c2.append(b)
        nCX(k,c1,b,nch+1)
        nCX(k,c2,t,nch-1)
        nCX(k,c1,b,nch+1)
        nCX(k,c2,t,nch-1)
    return

In [46]:
def showQasm():
    file=open("test_output/qpm_a4.qasm","r")
    print("\n~~~~~CODEFILE~~~~~\n")
    for line in file:
        print(line,end='')
    print()
    file.close()


In [56]:
if __name__=='__main__':
    QPM()

33331032233 331
Traceback (most recent call last):
  File "<ipython-input-56-b3e1b431c575>", line 2, in <module>
    QPM()
  File "<ipython-input-55-dd81473d1629>", line 4, in QPM
    platform=ql.Platform('platform_none',config_fn)
NameError: name 'ql' is not defined

Use %tb to get the full traceback.
